# Variables

## Import des librairies

In [ ]:
import os
import sys
from rdflib import Graph, Namespace, URIRef, Literal, BNode, XSD

## Définition des variables globales

In [ ]:
# Fichiers existants
ont_file_name = "ontology.ttl" # Fusion des deux fichiers précédents
ruleset_file_name = "rules.pie"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"
facts_ttl_file_name = "facts_data.ttl"
implicit_to_facts_ttl_file_name = "implicit_to_facts.ttl"

# Dossiers existants
data_folder_name = "data"
mapping_folder_name = "mappings"

# Dossiers créés durant le traitement
tmp_folder_name = "tmp_files"

# Nom du répertoire GraphDB pour les faits
facts_repository_name = "addresses_from_factoids_6"

# Définition des graphes nommés
ontology_named_graph_name = "ontology"
facts_named_graph_name = "facts"
factoids_named_graph_name = "factoids"
permanent_named_graph_name = "permanent"
tmp_named_graph_name = "temporary"

# Définition de namespaces et de préfixes associés
namespace_prefixes = {"addr":Namespace("http://rdf.geohistoricaldata.org/def/address#"),
                      "atype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/attributeType/"),
                      "ltype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/"),
                      "lrtype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/"),
                      "ctype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/changeType/"),
                      "facts":Namespace("http://rdf.geohistoricaldata.org/id/address/facts/"),
                      "factoids":Namespace("http://rdf.geohistoricaldata.org/id/address/factoids/"),
                      "geo": Namespace("http://www.opengis.net/ont/geosparql#"),
                      "skos": Namespace("http://www.w3.org/2004/02/skos/core#"),
                      "rdf": Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#"),
                      "rdfs": Namespace("http://www.w3.org/2000/01/rdf-schema#"),
                      }

# URIs pour accéder aux logiciels GraphDB et Ontotext-Refine
graphdb_url = "http://localhost:7200"
ontorefine_url = "http://localhost:7333"

# Commande pour lancer `Ontorefine`, dépend de l'OS et d'où il se situe
# ontorefine_cmd = "ontorefine-cli" # Nom sans chemin
# ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli" #Ubuntu
ontorefine_cmd = "/Applications/Ontotext\\ Refine.app/Contents/app/bin/ontorefine-cli" #MacOS

py_code_folder_path = "../code"

## Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)
facts_ttl_file = os.path.join(tmp_folder, facts_ttl_file_name)
implicit_to_facts_ttl_file = os.path.join(tmp_folder, implicit_to_facts_ttl_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import wikidata as wd
import ontorefine as otr
import graphdb as gd
import graphrdf as gr
import strprocessing as sp
import geomprocessing as gp
import multisourcesprocessing as msp
import factoidscreation as fc

## Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait.

In [ ]:
# Il se peut que la suppression d'un répertoire ne fonctionne pas donc pour éviter la suppresion au moment de la réinitialisation (suppression + (re)création)
# `allow_removal` doit valoir False et dans ce cas-là, le répertoire sera juste vidé.
allow_removal = False
disable_same_as = False
# gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, ruleset_file="owl2-rl-optimized", disable_same_as=disable_same_as, allow_removal=allow_removal)

gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, ruleset_file=ruleset_file, disable_same_as=disable_same_as, allow_removal=allow_removal)
# gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, disable_same_as=disable_same_as, allow_removal=allow_removal)

## Import des ontologies

In [ ]:
gd.load_ontologies(graphdb_url, facts_repository_name, [ont_file], ontology_named_graph_name)

## Définition de variables liées aux sources

### Voies de Paris via Wikidata

* `wd` pour "wikidata"
* `wdpt` pour "wikidata paris thoroughfares"
* `wdpa` pour "wikidata paris areas"
* `wdpl` pour "wikidata paris thoroughfare locations"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de Wikidata
wd_repository_name = "factoids_wikidata"

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpt_csv_file_name = "wd_paris_thoroughfares.csv"
wdpt_csv_file = os.path.join(data_folder, wdpt_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpt_mapping_file_name = "mapping_wikidata_landmarks_thoroughfares.json"
wdpt_mapping_file = os.path.join(mapping_folder, wdpt_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
wdpt_kg_file_name = "wd_paris_thoroughfares.ttl"
wdpt_kg_file = os.path.join(tmp_folder, wdpt_kg_file_name)

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpa_csv_file_name = "wd_paris_areas.csv"
wdpa_csv_file = os.path.join(data_folder, wdpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpa_mapping_file_name = "mapping_wikidata_landmarks_areas.json"
wdpa_mapping_file = os.path.join(mapping_folder, wdpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des zones de Paris
wdpa_kg_file_name = "wd_paris_areas.ttl"
wdpa_kg_file = os.path.join(tmp_folder, wdpa_kg_file_name)

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpl_csv_file_name = "wd_paris_locations.csv"
wdpl_csv_file = os.path.join(data_folder, wdpl_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpl_mapping_file_name = "mapping_wikidata_landmark_relations.json"
wdpl_mapping_file = os.path.join(mapping_folder, wdpl_mapping_file_name)

# Fichier TTL pour structurer les connaissances des zones de Paris
wdpl_kg_file_name = "wd_paris_thoroughfare_locations.ttl"
wdpl_kg_file = os.path.join(tmp_folder, wdpl_kg_file_name)

# Fichier TTL final des factoïdes de Wikidata
wd_factoids_kg_file_name = "wd_factoids.ttl"
wd_factoids_kg_file = os.path.join(tmp_folder, wd_factoids_kg_file_name)
wd_permanent_kg_file_name = "wd_permanent.ttl"
wd_permanent_kg_file = os.path.join(tmp_folder, wd_permanent_kg_file_name)

### Nomenclature des voies de la ville de Paris

Les données de la ville de Paris sont composées de deux jeux :
* [dénominations des emprises des voies actuelles](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [dénominations caduques des voies](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Les voies actuelles ont une emprise géométrique contrairement aux anciennes voies.

* `vpt` pour "ville paris thoroughfares"
* `vpta` pour "ville paris thoroughfares actuelles"
* `vptc` pour "ville paris thoroughfares caduques"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de la BAN
vpt_repository_name = "factoids_ville_de_paris"

# Fichier CSV pour stocker le résultat de la requête de la sélection
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(data_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(data_folder, vptc_csv_file_name)

# Propriétés liées au fichier `vpta_csv_file` (colonne décrivant la géométrie, séparateur de colonnes...)
vpta_csv_file_delimiter = ";"
vpta_csv_file_geom_col_name = "geo_shape"

# Le fichier `vpta_csv_file_name` sera modifié durant le traitement. Le fichier ci-dessous sera la version modifiée.
vpta_mod_csv_file_name = "denominations-emprises-voies-actuelles_wkt.csv"
vpta_mod_csv_file = os.path.join(tmp_folder, vpta_mod_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
vpta_mapping_file_name = "mapping_voies_paris_actuelles.json"
vpta_mapping_file = os.path.join(mapping_folder, vpta_mapping_file_name)
vptc_mapping_file_name = "mapping_voies_paris_caduques.json"
vptc_mapping_file = os.path.join(mapping_folder, vptc_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
vpta_kg_file_name = "voies_paris_actuelles.ttl"
vpta_kg_file = os.path.join(tmp_folder, vpta_kg_file_name)
vptc_kg_file_name = "voies_paris_caduques.ttl"
vptc_kg_file = os.path.join(tmp_folder, vptc_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
vpt_factoids_kg_file_name = "vpt_factoids.ttl"
vpt_factoids_kg_file = os.path.join(tmp_folder, vpt_factoids_kg_file_name)
vpt_permanent_kg_file_name = "vpt_permanent.ttl"
vpt_permanent_kg_file = os.path.join(tmp_folder, vpt_permanent_kg_file_name)

### Base Adresse Nationale (BAN)

Données de la [Base Adresse Nationale (BAN)](https://adresse.data.gouv.fr/base-adresse-nationale), accessible [ici](https://adresse.data.gouv.fr/data/ban/adresses/latest/csv)

`bpa` pour "BAN paris addresses"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de la BAN
bpa_repository_name = "factoids_ban"

# Fichier CSV pour stocker le résultat de la requête de la sélection
bpa_csv_file_name = "ban_adresses.csv"
bpa_csv_file = os.path.join(data_folder, bpa_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
bpa_mapping_file_name = "mapping_ban_adresses.json"
bpa_mapping_file = os.path.join(mapping_folder, bpa_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
bpa_kg_file_name = "ban_adresses.ttl"
bpa_kg_file = os.path.join(tmp_folder, bpa_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
bpa_factoids_kg_file_name = "ban_factoids.ttl"
bpa_factoids_kg_file = os.path.join(tmp_folder, bpa_factoids_kg_file_name)
bpa_permanent_kg_file_name = "ban_permanent.ttl"
bpa_permanent_kg_file = os.path.join(tmp_folder, bpa_permanent_kg_file_name)

### OpenStreetMap (OSM)

Extraction des données d'OpenStreetMap

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données d'OSM
osm_repository_name = "factoids_osm"

# Fichiers CSV pour stocker les résultats des requêtes de la sélection
osm_csv_file_name = "osm_adresses.csv"
osm_csv_file = os.path.join(data_folder, osm_csv_file_name)
osm_hn_csv_file_name = "osm_hn_adresses.csv"
osm_hn_csv_file = os.path.join(data_folder, osm_hn_csv_file_name)

# Fichiers de mapping pour transformer les fichier CSV en fichier TTL
osm_mapping_file_name = "mapping_osm_adresses.json"
osm_mapping_file = os.path.join(mapping_folder, osm_mapping_file_name)
osm_hn_mapping_file_name = "mapping_osm_hn_adresses.json"
osm_hn_mapping_file = os.path.join(mapping_folder, osm_hn_mapping_file_name)

# Fichiers TTL pour structurer les connaissances venant d'OSM
osm_kg_file_name = "osm_adresses.ttl"
osm_kg_file = os.path.join(tmp_folder, osm_kg_file_name)
osm_hn_kg_file_name = "osm_adresses_hn.ttl"
osm_hn_kg_file = os.path.join(tmp_folder, osm_hn_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
osm_factoids_kg_file_name = "osm_factoids.ttl"
osm_factoids_kg_file = os.path.join(tmp_folder, osm_factoids_kg_file_name)
osm_permanent_kg_file_name = "osm_permanent.ttl"
osm_permanent_kg_file = os.path.join(tmp_folder, osm_permanent_kg_file_name)

## Processus final et itératif

### Création des factoïdes dans des répertoires

Pour chaque source, des factoïdes sont créés indépendamment dans des répertoires distincts

#### Ville de Paris


In [ ]:
# fc.create_factoid_process_ville_paris(graphdb_url, vpt_repository_name, namespace_prefixes, tmp_folder,
#                                       ontorefine_url, ontorefine_cmd, ont_file, ontology_named_graph_name, factoids_named_graph_name, permanent_named_graph_name,
#                                       vpta_csv_file, vpta_mod_csv_file, vpta_csv_file_delimiter, vpta_csv_file_geom_col_name,
#                                       vptc_csv_file, vpta_mapping_file, vptc_mapping_file, vpta_kg_file, vptc_kg_file)

####  BAN


In [ ]:
# fc.create_factoid_process_ban(graphdb_url, bpa_repository_name, namespace_prefixes, tmp_folder,
#                               ontorefine_url, ontorefine_cmd,
#                               ont_file, ontology_named_graph_name, factoids_named_graph_name, permanent_named_graph_name,
#                               bpa_csv_file, bpa_mapping_file, bpa_kg_file)

#### Wikidata


In [ ]:
# # fc.get_data_from_wikidata(wdpt_csv_file, wdpa_csv_file, wdpl_csv_file)
# fc.create_factoid_process_wikidata(graphdb_url, wd_repository_name, namespace_prefixes, tmp_folder,
#                                    ontorefine_url, ontorefine_cmd,
#                                    ont_file, ontology_named_graph_name, factoids_named_graph_name, permanent_named_graph_name,
#                                    wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file,
#                                    wdpa_csv_file, wdpa_mapping_file, wdpa_kg_file,
#                                    wdpl_csv_file, wdpl_mapping_file, wdpl_kg_file)

#### OSM

In [ ]:
# fc.create_factoid_process_osm(graphdb_url, osm_repository_name, namespace_prefixes, tmp_folder,
#                               ontorefine_url, ontorefine_cmd, ont_file, ontology_named_graph_name, factoids_named_graph_name, permanent_named_graph_name,
#                               osm_csv_file, osm_mapping_file, osm_kg_file,
#                               osm_hn_csv_file, osm_hn_mapping_file, osm_hn_kg_file)

### Insertion des factoïdes dans le graphe des faits

#### Ville de Paris

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, vpt_repository_name, vpt_factoids_kg_file, vpt_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

#### Wikidata

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, wd_repository_name, wd_factoids_kg_file, wd_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

#### BAN

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, bpa_repository_name, bpa_factoids_kg_file, bpa_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

#### OSM

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, osm_repository_name, osm_factoids_kg_file, osm_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

## Intégration de données issues de fichiers Geojson

Ces fichiers sont issus de la vectorisation de plans de Paris
* cadatre napoléonien révisé de 1847
* plan "Andriveau" de 1849
* plan parcellaire municipal de 1871
* plan de l'Atlas Municipal de 1888

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
cn_1847_repository_name = "factoids_1847_cadastre_nap"

cn_1847_geojson_file_name = "1847_cadastre_nap.geojson"
cn_1847_geojson_file = os.path.join(data_folder, cn_1847_geojson_file_name)
cn_1847_kg_file_name = "cn_1847_kg.ttl"
cn_1847_kg_file = os.path.join(tmp_folder, cn_1847_kg_file_name)

# Fichier TTL final des factoïdes du cadatre napoléonien révisé de 1847
cn_1847_factoids_kg_file_name = "cn_1847_factoids.ttl"
cn_1847_factoids_kg_file = os.path.join(tmp_folder, cn_1847_factoids_kg_file_name)
cn_1847_permanent_kg_file_name = "cn_1847_permanent.ttl"
cn_1847_permanent_kg_file = os.path.join(tmp_folder, cn_1847_permanent_kg_file_name)

# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
an_1849_repository_name = "factoids_1849_andriveau"

an_1849_geojson_file_name = "1849_andriveau.geojson"
an_1849_geojson_file = os.path.join(data_folder, an_1849_geojson_file_name)
an_1849_kg_file_name = "an_1849_kg.ttl"
an_1849_kg_file = os.path.join(tmp_folder, an_1849_kg_file_name)

# Fichier TTL final des factoïdes du plan "Andriveau" de 1849
an_1849_factoids_kg_file_name = "an_1849_factoids.ttl"
an_1849_factoids_kg_file = os.path.join(tmp_folder, an_1849_factoids_kg_file_name)
an_1849_permanent_kg_file_name = "an_1849_permanent.ttl"
an_1849_permanent_kg_file = os.path.join(tmp_folder, an_1849_permanent_kg_file_name)

# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
pm_1871_repository_name = "factoids_1871_plan_parcellaire_mun"

pm_1871_geojson_file_name = "1871_plan_parcellaire_mun.geojson"
pm_1871_geojson_file = os.path.join(data_folder, pm_1871_geojson_file_name)
pm_1871_kg_file_name = "pm_1871_kg.ttl"
pm_1871_kg_file = os.path.join(tmp_folder, pm_1871_kg_file_name)

# Fichier TTL final des factoïdes du plan parcellaire municipal de 1871
pm_1871_factoids_kg_file_name = "pm_1871_factoids.ttl"
pm_1871_factoids_kg_file = os.path.join(tmp_folder, pm_1871_factoids_kg_file_name)
pm_1871_permanent_kg_file_name = "pm_1871_permanent.ttl"
pm_1871_permanent_kg_file = os.path.join(tmp_folder, pm_1871_permanent_kg_file_name)

# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
am_1888_repository_name = "factoids_1888_atlas_municipal"

am_1888_geojson_file_name = "1888_atlas_municipal.geojson"
am_1888_geojson_file = os.path.join(data_folder, am_1888_geojson_file_name)
am_1888_kg_file_name = "am_1888_kg.ttl"
am_1888_kg_file = os.path.join(tmp_folder, am_1888_kg_file_name)

# Fichier TTL final des factoïdes du plan de l'Atlas Municipal de 1888
am_1888_factoids_kg_file_name = "am_1888_factoids.ttl"
am_1888_factoids_kg_file = os.path.join(tmp_folder, am_1888_factoids_kg_file_name)
am_1888_permanent_kg_file_name = "am_1888_permanent.ttl"
am_1888_permanent_kg_file = os.path.join(tmp_folder, am_1888_permanent_kg_file_name)

In [ ]:
def create_landmark_from_geojson_feature(feature:dict, landmark_type:str, g:Graph, namespace_prefixes:dict, srs_uri:URIRef=None, lang:str=None):
    label = feature.get("properties").get("name")

    if srs_uri is None:
        geometry = gp.from_geojson_to_wkt(feature.get("geometry"))
    else:
        geometry = srs_uri.n3() + " " + gp.from_geojson_to_wkt(feature.get("geometry"))

    geometry_lit = Literal(geometry, datatype=namespace_prefixes["geo"]["wktLiteral"])
    landmark_uri = gr.generate_uri(namespace_prefixes["factoids"], "LM")

    gr.create_landmark(landmark_uri, label, lang, landmark_type, g, namespace_prefixes["addr"], namespace_prefixes["ltype"])
    g.add((landmark_uri, namespace_prefixes["geo"]["asWKT"], geometry_lit))

    if label is not None:
        alt_label = sp.remove_abbreviations_in_french_street_name(label)
        alt_label_lit = Literal(alt_label, lang=lang)
        g.add((landmark_uri, namespace_prefixes["skos"]["altLabel"], alt_label_lit))
    
    # landmark_uri = gr.generate_uri(namespace_prefixes["factoids"], "LM")
    # name_attribute_uri = gr.generate_uri(namespace_prefixes["factoids"], "ATTR")
    # geom_attribute_uri = gr.generate_uri(namespace_prefixes["factoids"], "ATTR")
    
    # gr.create_landmark_with_changes(landmark_uri, name_version, lang, landmark_type, g,
    #                              namespace_prefixes["addr"], namespace_prefixes["factoids"], namespace_prefixes["ltype"], namespace_prefixes["ctype"])
    # gr.create_landmark_attribute(name_attribute_uri, landmark_uri, "Name", g, namespace_prefixes["addr"], namespace_prefixes["atype"])
    # gr.create_landmark_attribute(geom_attribute_uri, landmark_uri, "Geometry", g, namespace_prefixes["addr"], namespace_prefixes["atype"])
    # gr.create_attribute_version(name_attribute_uri, name_version, g, namespace_prefixes["addr"], namespace_prefixes["factoids"], namespace_prefixes["ctype"], lang=lang)
    # gr.create_attribute_version(geom_attribute_uri, geom_version, g, namespace_prefixes["addr"], namespace_prefixes["factoids"], namespace_prefixes["ctype"], datatype=geom_datatype)

def create_landmarks_from_geojson_feature_collection(feature_collection:dict, landmark_type:str, g:Graph, namespace_prefixes:dict, lang:str=None):
    crs_dict = {
        "urn:ogc:def:crs:OGC:1.3:CRS84" : URIRef("http://www.opengis.net/def/crs/EPSG/0/4326"), 
        "urn:ogc:def:crs:EPSG::2154" :  URIRef("http://www.opengis.net/def/crs/EPSG/0/2154"), 
    }

    features = feature_collection.get("features")
    geojson_crs = feature_collection.get("crs")
    srs_iri = get_srs_iri_from_geojson_feature_collection(geojson_crs, crs_dict)
    
    for feature in features:
        create_landmark_from_geojson_feature(feature, landmark_type, g, namespace_prefixes, srs_iri, lang=lang)

def get_srs_iri_from_geojson_feature_collection(geojson_crs, crs_dict):  
    try:
        crs_name = geojson_crs.get("properties").get("name")
        srs_iri = crs_dict.get(crs_name)
        return srs_iri
    except:
        return None
    
def create_landmark_graph_from_geojson_file(geojson_file:str,out_kg_file, landmark_type:str,namespace_prefixes:dict,lang:str=None):
    feature_collection = fm.read_json_file(geojson_file)
    g = Graph()
    create_landmarks_from_geojson_feature_collection(feature_collection, landmark_type, g, namespace_prefixes, lang)
    g.serialize(out_kg_file)

In [ ]:
def create_factoid_process_geojson(graphdb_url, repository_name, namespace_prefixes, tmp_folder,
                                   ont_file, ontology_named_graph_name,
                                   factoids_named_graph_name, permanent_named_graph_name,
                                   geojson_file, kg_file, landmark_type, lang:str=None):

    """
    Fonction pour faire l'ensemble des processus relatifs à la création des factoïdes pour les données issues d'un fichier Geojson
    """

    # Création d'un répertoire des factoïdes pour les données
    msp.create_factoid_repository(graphdb_url, repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name, disable_same_as=False, clear_if_exists=True)

    # Récupération des URI des graphes nommés
    factoids_named_graph_uri = URIRef(gd.get_named_graph_uri_from_name(graphdb_url, repository_name, factoids_named_graph_name))
    permanent_named_graph_uri = URIRef(gd.get_named_graph_uri_from_name(graphdb_url, repository_name, permanent_named_graph_name))

    # À partir du fichier geojson décrivant des repères (qui sont tous du même type), convertir en un graphe de connaissance selon l'ontologie
    create_landmark_graph_from_geojson_file(geojson_file, kg_file, landmark_type, namespace_prefixes, lang)

    # Importer le fichier `kg_file` qui a été créé lors de la ligne précédente dans le répertoire `repository_name`, dans le graphe nommé `factoids_named_graph_name` 
    gd.import_ttl_file_in_graphdb(graphdb_url, repository_name, kg_file, factoids_named_graph_name)

    # # Suppression des instants qui n'ont aucun timeStamp (instants sans date)
    # msp.remove_time_instant_without_timestamp(graphdb_url, repository_name)

    # Nettoyer les données en fusionnant les doublons après l'import dans GraphDB
    clean_geojson_graph(graphdb_url, repository_name, factoids_named_graph_uri, permanent_named_graph_uri)

    # Ajout d'éléments manquants
    msp.add_missing_elements_for_landmarks(graphdb_url, repository_name, factoids_named_graph_uri)

    # # L'URI ci-dessous définit la source liée à la BAN
    # ban_factoids_uri = URIRef("http://rdf.geohistoricaldata.org/id/address/facts/Source_BAN")
    # create_source_ban(graphdb_url, repository_name, ban_factoids_uri, permanent_named_graph_uri)

    # Ajout de labels normalisés
    msp.add_normalized_label_for_landmarks(graphdb_url, repository_name, factoids_named_graph_uri)

    # Transfert de triplets non modifiables vers le graphe nommé permanent
    msp.transfert_immutable_triples(graphdb_url, repository_name, factoids_named_graph_uri, permanent_named_graph_uri)
    
    # # Ajout de liens entre les ressources de type repère et la source
    # msp.add_factoids_resources_links(graphdb_url, repository_name, factoids_named_graph_uri)

def clean_geojson_graph(graphdb_url, repository_name, factoids_named_graph_uri, permanent_named_graph_uri):
    prefixes = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
    PREFIX geofla: <http://data.ign.fr/def/geofla#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
    """

    # Suppresion des numéros d'immeubles qui appartiennent à aucune voie
    query1 = prefixes + f"""
    DELETE {{
            ?hn ?p ?o.
            ?s ?p ?hn.
        }}
    WHERE {{
        ?hn a addr:Landmark; addr:isLandmarkType ltype:HouseNumber.
        MINUS {{?lr a addr:LandmarkRelation; addr:locatum ?hn; addr:relatum [a addr:Landmark; addr:isLandmarkType ?thoroughfare] }}
        {{?hn ?p ?o.}}UNION{{?s ?p ?hn.}}
    }}
    """
    
    queries = [query1]
    for query in queries:
        gd.update_query(query, graphdb_url, repository_name)

In [ ]:
lang = "fr"
landmark_type = "Thoroughfare"

# create_factoid_process_geojson(graphdb_url, cn_1847_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
#                                factoids_named_graph_name, permanent_named_graph_name, cn_1847_geojson_file, cn_1847_kg_file, landmark_type, lang)
# create_factoid_process_geojson(graphdb_url, an_1849_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
#                                factoids_named_graph_name, permanent_named_graph_name, an_1849_geojson_file, an_1849_kg_file, landmark_type, lang)
# create_factoid_process_geojson(graphdb_url, pm_1871_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
#                                factoids_named_graph_name, permanent_named_graph_name, pm_1871_geojson_file, pm_1871_kg_file, landmark_type, lang)
# create_factoid_process_geojson(graphdb_url, am_1888_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
#                                factoids_named_graph_name, permanent_named_graph_name, am_1888_geojson_file, am_1888_kg_file, landmark_type, lang)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, cn_1847_repository_name,
                                       cn_1847_factoids_kg_file, cn_1847_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, an_1849_repository_name,
                                       an_1849_factoids_kg_file, an_1849_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, pm_1871_repository_name,
                                       pm_1871_factoids_kg_file, pm_1871_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, am_1888_repository_name,
                                       am_1888_factoids_kg_file, am_1888_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, factoids_named_graph_name, facts_named_graph_name, tmp_named_graph_name, facts_ttl_file, implicit_to_facts_ttl_file, ont_file, ontology_named_graph_name)


In [ ]:
import json
import geojson
from shapely.geometry import shape
from uuid import uuid4
feature_collection = fm.read_json_file(pm_1871_geojson_file)

def merge_geojson_features_from_one_property(feature_collection, property_name:str):
    """
    Merge all features of a geojson object which have the same property (name for instance)
    """

    new_geojson_features = []
    features_to_merge = {}
    
    for feat in feature_collection.get("features"):
        # Get property value for the feature
        property_value = feat.get("properties").get(property_name)

        # If the value is blank or does not exist, generate an uuid
        if property_value in [None, ""]:
            empty_value = True
            property_value = uuid4().hex
            feature_template = {"type":"Feature", "properties":{property_name:""}}
        else:
            empty_value = False
            feature_template = {"type":"Feature", "properties":{property_name:property_value}}

        features_to_merge_key = features_to_merge.get(property_value)

        if features_to_merge_key is None:
            features_to_merge[property_value] = [feature_template, [feat]]
        else:
            features_to_merge[features_to_merge_key][1].append(feat)

        print(features_to_merge)
    for elem in features_to_merge.values():
        template, feature = elem

        geom_collection_list = []
        for portion in feature:
            geom_collection_list.append(portion.get("geometry"))
    
        geom_collection = {"type": "GeometryCollection", "geometries": geom_collection_list}
        template["geometry"] = geom_collection
        new_geojson_features.append(template)

    new_geojson = {"type":"FeatureCollection", "features":new_geojson_features}

    return new_geojson

a = merge_geojson_features_from_one_property(feature_collection, "name")
print(a)